## 0. 학습 세팅

### 1) 메모리 정리

In [1]:
import gc
import torch

gc.collect()
torch.cuda.empty_cache()

### 2) 수정된 코드 자동 업데이트

In [2]:
%load_ext autoreload
%autoreload 2
import foolbox as fb

## 1. Load library

In [3]:
# 라이브러리 호출
import os
import time

import argparse
import easydict

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from torch.autograd import Variable

from tqdm import tqdm_notebook

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split  

## 2. Variable Declaration

In [4]:
# # Jupyter 외 환경
# parser = argparse.ArgumentParser()
# parser.add_argument("--title", type=str, default="baseline")
# parser.add_argument("--device", type=str, default="cuda")
# parser.add_argument("--root", type=str, default="data")
# parser.add_argument("--batch_size", type=int, default=64)
# parser.add_argument("--num_workers", type=int, default=2)
# parser.add_argument("--epochs", type=int, default=100)
# parser.add_argument("--lr", type=float, default=0.001)
# parser.add_argument("--logs", type=str, default='logs')
# parser.add_argument("--checkpoints", type=str, default='checkpoints')
# parser.add_argument("--resume", type=bool, default=False)
# args = parser.parse_args()

# Jupyter 환경
args = easydict.EasyDict({
        "title" : "VEEM_Test",
        "device" : "cuda",
        "root" : "data",
        "batch_size" : 32, # !!!
        "num_workers" : 2,
        "epochs" : 1000, # !!!
        "lr" : 0.0001, # !!!
        "logs" : "logs",
        "checkpoints" : "checkpoints",
        "resume" : False,
        "input_size" : 11,
        "hidden_size" : 2,
        "output_size" : 5,
        "num_layers" : 1,
    })

## 3. Data Preprocessing

### 2) 데이터셋 불러오기

In [5]:
# 데이터셋 불러오기 + 출력
Person_path = f'C:\\Users\\Bang\\JupyterProjects\\VEEM_Project\\data\\Person_SNSB\\Person.csv'
SNSB_path = f'C:\\Users\\Bang\\JupyterProjects\\VEEM_Project\\data\\Person_SNSB\\SNSB.csv'

Person_dataset=pd.read_csv(Person_path)
SNSB_dataset=pd.read_csv(SNSB_path)

print("ㅡㅡㅡㅡㅡ[Person_dataset.dtypes]ㅡㅡㅡㅡㅡ")
print(Person_dataset.dtypes)
print("ㅡㅡㅡㅡㅡ[SNSB_dataset.dtypes]ㅡㅡㅡㅡㅡ")
print(SNSB_dataset.dtypes)

ㅡㅡㅡㅡㅡ[Person_dataset.dtypes]ㅡㅡㅡㅡㅡ
번호        int64
성명 코드    object
집단       object
성별       object
나이        int64
교육연한      int64
dtype: object
ㅡㅡㅡㅡㅡ[SNSB_dataset.dtypes]ㅡㅡㅡㅡㅡ
번호                      int64
성명 코드                  object
집단                     object
DST_F+B                 int64
S-K-BNT                 int64
RCFT_copyscore        float64
SVLT_delayedrecall      int64
K-TMT-E_B               int64
dtype: object


### 3) 데이터 자료형 변환

In [6]:
Person_dataset['나이'] = Person_dataset['나이'].astype(float)
Person_dataset['교육연한'] = Person_dataset['교육연한'].astype(float)

SNSB_dataset['DST_F+B'] = SNSB_dataset['DST_F+B'].astype(float)
SNSB_dataset['S-K-BNT'] = SNSB_dataset['S-K-BNT'].astype(float)
SNSB_dataset['SVLT_delayedrecall'] = SNSB_dataset['SVLT_delayedrecall'].astype(float)
SNSB_dataset['K-TMT-E_B'] = SNSB_dataset['K-TMT-E_B'].astype(float)

print("ㅡㅡㅡㅡㅡ[Person_dataset.dtypes]ㅡㅡㅡㅡㅡ")
print(Person_dataset.dtypes)
print("ㅡㅡㅡㅡㅡ[SNSB_dataset.dtypes]ㅡㅡㅡㅡㅡ")
print(SNSB_dataset.dtypes)

ㅡㅡㅡㅡㅡ[Person_dataset.dtypes]ㅡㅡㅡㅡㅡ
번호         int64
성명 코드     object
집단        object
성별        object
나이       float64
교육연한     float64
dtype: object
ㅡㅡㅡㅡㅡ[SNSB_dataset.dtypes]ㅡㅡㅡㅡㅡ
번호                      int64
성명 코드                  object
집단                     object
DST_F+B               float64
S-K-BNT               float64
RCFT_copyscore        float64
SVLT_delayedrecall    float64
K-TMT-E_B             float64
dtype: object


### 4) 인덱스 지정

In [7]:
Person_dataset.set_index('번호', inplace=True)
SNSB_dataset.set_index('번호', inplace=True)

In [8]:
print("ㅡㅡㅡㅡㅡ[Person_dataset]ㅡㅡㅡㅡㅡ")
Person_dataset

ㅡㅡㅡㅡㅡ[Person_dataset]ㅡㅡㅡㅡㅡ


,성명 코드,집단,성별,나이,교육연한
번호,,,,,
2,NSU1,HC,여성,71.0,16.0
3,CSR1,HC,남성,73.0,16.0
4,KMO1,MCI,여성,59.0,12.0
6,LCH1,MCI,남성,83.0,14.0
7,LYH1,MCI,여성,67.0,15.0
8,MYG1,HC,남성,60.0,16.0
9,YHR1,HC,남성,79.0,16.0
10,LJG1,MCI,남성,69.0,12.0
11,PHG1,MCI,남성,86.0,12.0


In [9]:
print("ㅡㅡㅡㅡㅡ[SNSB_dataset]ㅡㅡㅡㅡㅡ")
SNSB_dataset

ㅡㅡㅡㅡㅡ[SNSB_dataset]ㅡㅡㅡㅡㅡ


,성명 코드,집단,DST_F+B,S-K-BNT,RCFT_copyscore,SVLT_delayedrecall,K-TMT-E_B
번호,,,,,,,
2,NSU1,HC,13.0,11.0,35.0,11.0,16.0
3,CSR1,HC,10.0,15.0,30.0,5.0,26.0
4,KMO1,MCI,4.0,11.0,35.0,1.0,50.0
6,LCH1,MCI,5.0,8.0,18.5,0.0,300.0
7,LYH1,MCI,12.0,10.0,36.0,7.0,36.0
8,MYG1,HC,11.0,13.0,36.0,6.0,20.0
9,YHR1,HC,11.0,13.0,34.0,7.0,32.0
10,LJG1,MCI,9.0,12.0,27.0,0.0,43.0
11,PHG1,MCI,8.0,9.0,28.0,0.0,90.0


### 5) 필요한 피쳐 추출

In [10]:
# 데이터셋 나누기
X=Person_dataset.iloc[:, 2:]
y=SNSB_dataset.iloc[:, 2:]

In [11]:
print("ㅡㅡㅡㅡㅡ[Person_dataset]ㅡㅡㅡㅡㅡ")
X

ㅡㅡㅡㅡㅡ[Person_dataset]ㅡㅡㅡㅡㅡ


,성별,나이,교육연한
번호,,,
2,여성,71.0,16.0
3,남성,73.0,16.0
4,여성,59.0,12.0
6,남성,83.0,14.0
7,여성,67.0,15.0
8,남성,60.0,16.0
9,남성,79.0,16.0
10,남성,69.0,12.0
11,남성,86.0,12.0


In [12]:
print("ㅡㅡㅡㅡㅡ[SNSB_dataset]ㅡㅡㅡㅡㅡ")
y

ㅡㅡㅡㅡㅡ[SNSB_dataset]ㅡㅡㅡㅡㅡ


,DST_F+B,S-K-BNT,RCFT_copyscore,SVLT_delayedrecall,K-TMT-E_B
번호,,,,,
2,13.0,11.0,35.0,11.0,16.0
3,10.0,15.0,30.0,5.0,26.0
4,4.0,11.0,35.0,1.0,50.0
6,5.0,8.0,18.5,0.0,300.0
7,12.0,10.0,36.0,7.0,36.0
8,11.0,13.0,36.0,6.0,20.0
9,11.0,13.0,34.0,7.0,32.0
10,9.0,12.0,27.0,0.0,43.0
11,8.0,9.0,28.0,0.0,90.0


### 2) 데이터셋 불러오기

In [13]:
# 데이터셋 불러오기 + 출력
eyerpt_path = f'C:\\Users\\Bang\\JupyterProjects\\VEEM_Project\\data\\rpt\\02_eyerpt.csv'
rpt_path = f'C:\\Users\\Bang\\JupyterProjects\\VEEM_Project\\data\\rpt\\02_rpt.csv'

eyerpt_dataset=pd.read_csv(eyerpt_path)
rpt_dataset=pd.read_csv(rpt_path)

print("ㅡㅡㅡㅡㅡ[eyerpt_dataset.dtypes]ㅡㅡㅡㅡㅡ")
print(eyerpt_dataset.dtypes)
print("ㅡㅡㅡㅡㅡ[rpt_dataset.dtypes]ㅡㅡㅡㅡㅡ")
print(rpt_dataset.dtypes)

ㅡㅡㅡㅡㅡ[eyerpt_dataset.dtypes]ㅡㅡㅡㅡㅡ
Panel_num       int64
time           object
beforeOBJ      object
presentOBJ     object
PanelOrNot      int64
               ...   
count_0         int64
ok            float64
ok_count        int64
error           int64
time stamp     object
Length: 98, dtype: object
ㅡㅡㅡㅡㅡ[rpt_dataset.dtypes]ㅡㅡㅡㅡㅡ
panel_num              float64
total_task_time_s       object
head_x_m               float64
head_y_m               float64
head_z_m               float64
head_x_rotation_deg    float64
head_y_rotation_deg    float64
head_z_rotation_deg    float64
hand_x_m               float64
hand_y_m               float64
hand_z_m               float64
hand_x_rotation_deg    float64
hand_y_rotation_deg    float64
hand_z_rotation_deg    float64
error                  float64
time stamp              object
dtype: object


### 3) 데이터 자료형 변환

rpt_dataset['panel_num'] = rpt_dataset['panel_num'].astype(float)
rpt_dataset['error'] = rpt_dataset['error'].astype(float)

print("ㅡㅡㅡㅡㅡ[eyerpt_dataset.dtypes]ㅡㅡㅡㅡㅡ")
print(eyerpt_dataset.dtypes)
print("ㅡㅡㅡㅡㅡ[rpt_dataset.dtypes]ㅡㅡㅡㅡㅡ")
print(rpt_dataset.dtypes)

### 4) 인덱스 지정

In [14]:
# eyerpt_dataset['time stamp'] = pd.to_datetime(eyerpt_dataset['time stamp'])
# rpt_dataset['time stamp'] = pd.to_datetime(rpt_dataset['time stamp'])
eyerpt_dataset.set_index('time stamp', inplace=True)
rpt_dataset.set_index('time stamp', inplace=True)

In [15]:
print("ㅡㅡㅡㅡㅡ[Person_dataset]ㅡㅡㅡㅡㅡ")
eyerpt_dataset

ㅡㅡㅡㅡㅡ[Person_dataset]ㅡㅡㅡㅡㅡ


,Panel_num,time,beforeOBJ,presentOBJ,PanelOrNot,Obeject_name,eyeOpennes_L,eyeOpennes_R,Panel_X,Panel_Y,...,count_8,n9,count_9,delete,delete_count,n0,count_0,ok,ok_count,error
time stamp,,,,,,,,,,,,,,,,,,,,,
2022-01-11 PM 5:49:57,0,time,PlayButton,PlayButton,1,PlayButton,0.488791,0.263064,-0.010966,1.378456,...,0,0.000000,0,0.000000,0,0.00000,0,0.00000,0,0
2022-01-11 PM 5:49:57,0,time,PlayButton,PlayButton,1,PlayButton,0.570827,0.319782,0.022528,1.330381,...,0,0.000000,0,0.000000,0,0.00000,0,0.00000,0,0
2022-01-11 PM 5:49:57,0,time,PlayButton,PlayButton,1,PlayButton,0.469254,0.213507,0.047224,1.317823,...,0,0.000000,0,0.000000,0,0.00000,0,0.00000,0,0
2022-01-11 PM 5:49:57,0,time,PlayButton,PlayButton,1,PlayButton,0.596795,0.340741,-0.045546,1.306427,...,0,0.000000,0,0.000000,0,0.00000,0,0.00000,0,0
2022-01-11 PM 5:49:57,0,time,PlayButton,PlayButton,1,PlayButton,0.551406,0.324814,0.011136,1.325926,...,0,0.000000,0,0.000000,0,0.00000,0,0.00000,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-11 PM 5:52:21,8,time,OkButton,Canvas,0,SM_FastFood_Chair_02 (1),1.000000,1.000000,-2.991918,0.853727,...,34,1.077766,21,0.244476,3,1.67778,21,0.67778,1,4
2022-01-11 PM 5:52:21,8,time,OkButton,Canvas,0,SM_FastFood_Lamp_03,0.661335,0.294476,-3.467731,2.635472,...,34,1.077766,21,0.244476,3,1.67778,21,0.67778,1,4
2022-01-11 PM 5:52:21,8,time,OkButton,Canvas,0,SM_FastFood_Lamp_03,0.000000,0.000000,-2.791847,2.499408,...,34,1.077766,21,0.244476,3,1.67778,21,0.67778,1,4


In [16]:
print("ㅡㅡㅡㅡㅡ[SNSB_dataset]ㅡㅡㅡㅡㅡ")
rpt_dataset

ㅡㅡㅡㅡㅡ[SNSB_dataset]ㅡㅡㅡㅡㅡ


,panel_num,total_task_time_s,head_x_m,head_y_m,head_z_m,head_x_rotation_deg,head_y_rotation_deg,head_z_rotation_deg,hand_x_m,hand_y_m,hand_z_m,hand_x_rotation_deg,hand_y_rotation_deg,hand_z_rotation_deg,error
time stamp,,,,,,,,,,,,,,,
2022-01-11 PM 5:49:48,0.0,NaN,0.000000,1.133000,0.450000,0.0000,0.0000,0.000000,0.250000,1.700000,1.000000,0.0000,0.0000,0.0000,0.0
2022-01-11 PM 5:49:49,0.0,time,-0.018086,1.414308,0.424859,356.0033,345.4337,2.793784,0.018513,1.394299,1.358461,314.6794,356.9407,359.9806,0.0
2022-01-11 PM 5:49:49,0.0,time,-0.018086,1.414308,0.424859,356.0033,345.4337,2.793784,0.018513,1.394299,1.358461,314.6794,356.9407,359.9806,0.0
2022-01-11 PM 5:49:49,0.0,time,-0.018086,1.414308,0.424859,356.0033,345.4337,2.793784,0.018513,1.394299,1.358461,314.6794,356.9407,359.9806,0.0
2022-01-11 PM 5:49:49,0.0,time,-0.018086,1.414308,0.424859,356.0033,345.4337,2.793784,0.018513,1.394299,1.358461,314.6794,356.9407,359.9806,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-11 PM 5:52:21,8.0,time,-0.124581,1.438099,-0.080958,320.5923,348.5423,29.406940,-0.615525,1.274826,2.006991,339.5294,247.4133,331.8916,4.0
2022-01-11 PM 5:52:21,8.0,time,-0.114827,1.437855,-0.073374,326.6092,349.1585,27.131180,-0.614905,1.272514,2.006251,339.4104,247.5626,331.3984,4.0
2022-01-11 PM 5:52:22,8.0,time,-0.104077,1.436943,-0.063789,331.3346,350.8531,24.333350,-0.615532,1.270251,2.005716,339.8100,247.2282,331.4058,4.0


### 5) 데이터 프레임 변환 + 필요한 피쳐 추출

In [17]:
# 데이터 프레임 변환
df_eyerpt = eyerpt_dataset.loc[:, :]
df_rpt = rpt_dataset.loc[:, :]

# 데이터 프레임 필요한 피쳐 추출
df_eyerpt = df_eyerpt.drop(['time', 'beforeOBJ', 'presentOBJ', 'Obeject_name'], axis = 1) # 1 = columns
df_rpt = df_rpt.drop(['total_task_time_s', 'hand_x_rotation_deg', 'hand_y_rotation_deg', 'hand_z_rotation_deg'], axis = 1) # 1= columns

In [18]:
print("ㅡㅡㅡㅡㅡ[df_eyerpt]ㅡㅡㅡㅡㅡ")
df_eyerpt

ㅡㅡㅡㅡㅡ[df_eyerpt]ㅡㅡㅡㅡㅡ


,Panel_num,PanelOrNot,eyeOpennes_L,eyeOpennes_R,Panel_X,Panel_Y,pupilDia_L,pupilDia_R,start,start_count,...,count_8,n9,count_9,delete,delete_count,n0,count_0,ok,ok_count,error
time stamp,,,,,,,,,,,,,,,,,,,,,
2022-01-11 PM 5:49:57,0,1,0.488791,0.263064,-0.010966,1.378456,1.922363,2.120407,0.033334,1,...,0,0.000000,0,0.000000,0,0.00000,0,0.00000,0,0
2022-01-11 PM 5:49:57,0,1,0.570827,0.319782,0.022528,1.330381,2.095856,2.146896,0.133333,1,...,0,0.000000,0,0.000000,0,0.00000,0,0.00000,0,0
2022-01-11 PM 5:49:57,0,1,0.469254,0.213507,0.047224,1.317823,2.279144,2.125412,0.233334,1,...,0,0.000000,0,0.000000,0,0.00000,0,0.00000,0,0
2022-01-11 PM 5:49:57,0,1,0.596795,0.340741,-0.045546,1.306427,2.179672,2.071182,0.333333,1,...,0,0.000000,0,0.000000,0,0.00000,0,0.00000,0,0
2022-01-11 PM 5:49:57,0,1,0.551406,0.324814,0.011136,1.325926,2.050919,2.235275,0.433333,1,...,0,0.000000,0,0.000000,0,0.00000,0,0.00000,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-11 PM 5:52:21,8,0,1.000000,1.000000,-2.991918,0.853727,3.058319,2.786301,26.700030,105,...,34,1.077766,21,0.244476,3,1.67778,21,0.67778,1,4
2022-01-11 PM 5:52:21,8,0,0.661335,0.294476,-3.467731,2.635472,-1.000000,-1.000000,26.700030,105,...,34,1.077766,21,0.244476,3,1.67778,21,0.67778,1,4
2022-01-11 PM 5:52:21,8,0,0.000000,0.000000,-2.791847,2.499408,-1.000000,-1.000000,26.700030,105,...,34,1.077766,21,0.244476,3,1.67778,21,0.67778,1,4


In [19]:
print("ㅡㅡㅡㅡㅡ[df_rpt]ㅡㅡㅡㅡㅡ")
df_rpt

ㅡㅡㅡㅡㅡ[df_rpt]ㅡㅡㅡㅡㅡ


,panel_num,head_x_m,head_y_m,head_z_m,head_x_rotation_deg,head_y_rotation_deg,head_z_rotation_deg,hand_x_m,hand_y_m,hand_z_m,error
time stamp,,,,,,,,,,,
2022-01-11 PM 5:49:48,0.0,0.000000,1.133000,0.450000,0.0000,0.0000,0.000000,0.250000,1.700000,1.000000,0.0
2022-01-11 PM 5:49:49,0.0,-0.018086,1.414308,0.424859,356.0033,345.4337,2.793784,0.018513,1.394299,1.358461,0.0
2022-01-11 PM 5:49:49,0.0,-0.018086,1.414308,0.424859,356.0033,345.4337,2.793784,0.018513,1.394299,1.358461,0.0
2022-01-11 PM 5:49:49,0.0,-0.018086,1.414308,0.424859,356.0033,345.4337,2.793784,0.018513,1.394299,1.358461,0.0
2022-01-11 PM 5:49:49,0.0,-0.018086,1.414308,0.424859,356.0033,345.4337,2.793784,0.018513,1.394299,1.358461,0.0
...,...,...,...,...,...,...,...,...,...,...,...
2022-01-11 PM 5:52:21,8.0,-0.124581,1.438099,-0.080958,320.5923,348.5423,29.406940,-0.615525,1.274826,2.006991,4.0
2022-01-11 PM 5:52:21,8.0,-0.114827,1.437855,-0.073374,326.6092,349.1585,27.131180,-0.614905,1.272514,2.006251,4.0
2022-01-11 PM 5:52:22,8.0,-0.104077,1.436943,-0.063789,331.3346,350.8531,24.333350,-0.615532,1.270251,2.005716,4.0


### 6) 결측치 행 제거

In [20]:
df_eyerpt.dropna(axis=0, inplace = True)
df_rpt.dropna(axis=0, inplace = True)

In [21]:
print("ㅡㅡㅡㅡㅡ[df_eyerpt]ㅡㅡㅡㅡㅡ")
df_eyerpt

ㅡㅡㅡㅡㅡ[df_eyerpt]ㅡㅡㅡㅡㅡ


,Panel_num,PanelOrNot,eyeOpennes_L,eyeOpennes_R,Panel_X,Panel_Y,pupilDia_L,pupilDia_R,start,start_count,...,count_8,n9,count_9,delete,delete_count,n0,count_0,ok,ok_count,error
time stamp,,,,,,,,,,,,,,,,,,,,,
2022-01-11 PM 5:49:57,0,1,0.488791,0.263064,-0.010966,1.378456,1.922363,2.120407,0.033334,1,...,0,0.000000,0,0.000000,0,0.00000,0,0.00000,0,0
2022-01-11 PM 5:49:57,0,1,0.570827,0.319782,0.022528,1.330381,2.095856,2.146896,0.133333,1,...,0,0.000000,0,0.000000,0,0.00000,0,0.00000,0,0
2022-01-11 PM 5:49:57,0,1,0.469254,0.213507,0.047224,1.317823,2.279144,2.125412,0.233334,1,...,0,0.000000,0,0.000000,0,0.00000,0,0.00000,0,0
2022-01-11 PM 5:49:57,0,1,0.596795,0.340741,-0.045546,1.306427,2.179672,2.071182,0.333333,1,...,0,0.000000,0,0.000000,0,0.00000,0,0.00000,0,0
2022-01-11 PM 5:49:57,0,1,0.551406,0.324814,0.011136,1.325926,2.050919,2.235275,0.433333,1,...,0,0.000000,0,0.000000,0,0.00000,0,0.00000,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-11 PM 5:52:21,8,0,1.000000,1.000000,-2.991918,0.853727,3.058319,2.786301,26.700030,105,...,34,1.077766,21,0.244476,3,1.67778,21,0.67778,1,4
2022-01-11 PM 5:52:21,8,0,0.661335,0.294476,-3.467731,2.635472,-1.000000,-1.000000,26.700030,105,...,34,1.077766,21,0.244476,3,1.67778,21,0.67778,1,4
2022-01-11 PM 5:52:21,8,0,0.000000,0.000000,-2.791847,2.499408,-1.000000,-1.000000,26.700030,105,...,34,1.077766,21,0.244476,3,1.67778,21,0.67778,1,4


In [22]:
print("ㅡㅡㅡㅡㅡ[df_rpt]ㅡㅡㅡㅡㅡ")
df_rpt

ㅡㅡㅡㅡㅡ[df_rpt]ㅡㅡㅡㅡㅡ


,panel_num,head_x_m,head_y_m,head_z_m,head_x_rotation_deg,head_y_rotation_deg,head_z_rotation_deg,hand_x_m,hand_y_m,hand_z_m,error
time stamp,,,,,,,,,,,
2022-01-11 PM 5:49:48,0.0,0.000000,1.133000,0.450000,0.0000,0.0000,0.000000,0.250000,1.700000,1.000000,0.0
2022-01-11 PM 5:49:49,0.0,-0.018086,1.414308,0.424859,356.0033,345.4337,2.793784,0.018513,1.394299,1.358461,0.0
2022-01-11 PM 5:49:49,0.0,-0.018086,1.414308,0.424859,356.0033,345.4337,2.793784,0.018513,1.394299,1.358461,0.0
2022-01-11 PM 5:49:49,0.0,-0.018086,1.414308,0.424859,356.0033,345.4337,2.793784,0.018513,1.394299,1.358461,0.0
2022-01-11 PM 5:49:49,0.0,-0.018086,1.414308,0.424859,356.0033,345.4337,2.793784,0.018513,1.394299,1.358461,0.0
...,...,...,...,...,...,...,...,...,...,...,...
2022-01-11 PM 5:52:21,8.0,-0.128660,1.438548,-0.084028,318.0778,348.0271,30.289050,-0.616032,1.278466,2.008882,4.0
2022-01-11 PM 5:52:21,8.0,-0.124581,1.438099,-0.080958,320.5923,348.5423,29.406940,-0.615525,1.274826,2.006991,4.0
2022-01-11 PM 5:52:21,8.0,-0.114827,1.437855,-0.073374,326.6092,349.1585,27.131180,-0.614905,1.272514,2.006251,4.0


In [23]:
df_eyerpt.Panel_num

time stamp
2022-01-11 PM 5:49:57    0
2022-01-11 PM 5:49:57    0
2022-01-11 PM 5:49:57    0
2022-01-11 PM 5:49:57    0
2022-01-11 PM 5:49:57    0
                        ..
2022-01-11 PM 5:52:21    8
2022-01-11 PM 5:52:21    8
2022-01-11 PM 5:52:21    8
2022-01-11 PM 5:52:22    8
2022-01-11 PM 5:52:22    8
Name: Panel_num, Length: 1447, dtype: int64

### 8) 종료 이후 데이터 제거

In [24]:
df_eyerpt_drop8 = df_eyerpt[df_eyerpt.Panel_num < 8]
df_rpt_drop8 = df_rpt[df_rpt.panel_num < 8]

In [25]:
print("ㅡㅡㅡㅡㅡ[df_eyerpt_drop8]ㅡㅡㅡㅡㅡ")
df_eyerpt_drop8

ㅡㅡㅡㅡㅡ[df_eyerpt_drop8]ㅡㅡㅡㅡㅡ


,Panel_num,PanelOrNot,eyeOpennes_L,eyeOpennes_R,Panel_X,Panel_Y,pupilDia_L,pupilDia_R,start,start_count,...,count_8,n9,count_9,delete,delete_count,n0,count_0,ok,ok_count,error
time stamp,,,,,,,,,,,,,,,,,,,,,
2022-01-11 PM 5:49:57,0,1,0.488791,0.263064,-0.010966,1.378456,1.922363,2.120407,0.033334,1,...,0,0.000000,0,0.000000,0,0.00000,0,0.000000,0,0
2022-01-11 PM 5:49:57,0,1,0.570827,0.319782,0.022528,1.330381,2.095856,2.146896,0.133333,1,...,0,0.000000,0,0.000000,0,0.00000,0,0.000000,0,0
2022-01-11 PM 5:49:57,0,1,0.469254,0.213507,0.047224,1.317823,2.279144,2.125412,0.233334,1,...,0,0.000000,0,0.000000,0,0.00000,0,0.000000,0,0
2022-01-11 PM 5:49:57,0,1,0.596795,0.340741,-0.045546,1.306427,2.179672,2.071182,0.333333,1,...,0,0.000000,0,0.000000,0,0.00000,0,0.000000,0,0
2022-01-11 PM 5:49:57,0,1,0.551406,0.324814,0.011136,1.325926,2.050919,2.235275,0.433333,1,...,0,0.000000,0,0.000000,0,0.00000,0,0.000000,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-11 PM 5:52:13,7,1,1.000000,1.000000,0.166148,1.314535,2.726440,2.617294,26.700030,105,...,34,1.077766,21,0.244476,3,1.67778,21,0.333328,1,4
2022-01-11 PM 5:52:14,7,1,1.000000,1.000000,0.158048,1.310377,2.725998,2.567093,26.700030,105,...,34,1.077766,21,0.244476,3,1.67778,21,0.433334,1,4
2022-01-11 PM 5:52:14,7,1,1.000000,1.000000,0.159890,1.309437,2.681747,2.563614,26.700030,105,...,34,1.077766,21,0.244476,3,1.67778,21,0.533325,1,4


In [26]:
print("ㅡㅡㅡㅡㅡ[df_rpt_drop8]ㅡㅡㅡㅡㅡ")
df_rpt_drop8

ㅡㅡㅡㅡㅡ[df_rpt_drop8]ㅡㅡㅡㅡㅡ


,panel_num,head_x_m,head_y_m,head_z_m,head_x_rotation_deg,head_y_rotation_deg,head_z_rotation_deg,hand_x_m,hand_y_m,hand_z_m,error
time stamp,,,,,,,,,,,
2022-01-11 PM 5:49:48,0.0,0.000000,1.133000,0.450000,0.000000,0.0000,0.000000,0.250000,1.700000,1.000000,0.0
2022-01-11 PM 5:49:49,0.0,-0.018086,1.414308,0.424859,356.003300,345.4337,2.793784,0.018513,1.394299,1.358461,0.0
2022-01-11 PM 5:49:49,0.0,-0.018086,1.414308,0.424859,356.003300,345.4337,2.793784,0.018513,1.394299,1.358461,0.0
2022-01-11 PM 5:49:49,0.0,-0.018086,1.414308,0.424859,356.003300,345.4337,2.793784,0.018513,1.394299,1.358461,0.0
2022-01-11 PM 5:49:49,0.0,-0.018086,1.414308,0.424859,356.003300,345.4337,2.793784,0.018513,1.394299,1.358461,0.0
...,...,...,...,...,...,...,...,...,...,...,...
2022-01-11 PM 5:52:13,7.0,0.014645,1.431674,-0.020886,7.709767,359.0674,2.388693,0.115171,1.303554,0.918626,4.0
2022-01-11 PM 5:52:14,7.0,0.013939,1.431054,-0.020546,7.111332,359.3663,2.534641,0.114009,1.309647,0.923092,4.0
2022-01-11 PM 5:52:14,7.0,0.013138,1.431253,-0.020841,6.653352,359.3607,2.557021,0.114875,1.311217,0.928992,4.0


### 9) 데이터 프레임 -> list 변환

In [50]:
df_eyerpt_drop8_np = df_eyerpt_drop8.to_numpy()
df_rpt_drop8_np = df_rpt_drop8.to_numpy()

In [56]:
print("ㅡㅡㅡㅡㅡ[df_eyerpt_drop8_np.shape]ㅡㅡㅡㅡㅡ")
print(df_eyerpt_drop8_np.shape)

print("ㅡㅡㅡㅡㅡ[df_rpt_drop8_np.shape]ㅡㅡㅡㅡㅡ")
print(df_rpt_drop8_np.shape)

ㅡㅡㅡㅡㅡ[df_eyerpt_drop8_np.shape]ㅡㅡㅡㅡㅡ
(1369, 93)
ㅡㅡㅡㅡㅡ[df_rpt_drop8_np.shape]ㅡㅡㅡㅡㅡ
(1458, 11)


In [59]:
print("ㅡㅡㅡㅡㅡ[df_eyerpt_drop8_np]ㅡㅡㅡㅡㅡ")
print(df_eyerpt_drop8_np)

print("ㅡㅡㅡㅡㅡ[df_rpt_drop8_np]ㅡㅡㅡㅡㅡ")
print(df_rpt_drop8_np)

ㅡㅡㅡㅡㅡ[df_eyerpt_drop8_np]ㅡㅡㅡㅡㅡ
[[0.        1.        0.4887909 ... 0.        0.        0.       ]
 [0.        1.        0.5708269 ... 0.        0.        0.       ]
 [0.        1.        0.4692542 ... 0.        0.        0.       ]
 ...
 [7.        1.        1.        ... 0.5333252 1.        4.       ]
 [7.        1.        1.        ... 0.6333313 1.        4.       ]
 [7.        1.        1.        ... 0.6777802 1.        4.       ]]


In [27]:
sequence_length = len(df_rpt_drop8)

print(sequence_length)

1458


In [28]:
"""
print(f"train_dataset 개수 : {len(train_dataset)}, " + f"train_loader 개수 : {len(train_loader)}") # train_data 개수, train_loader batch set 개수
print(f"val_dataset 개수 : {len(valid_dataset)}, " + f"val_loader 개수 : {len(val_loader)}") # val_data 개수, val_loader batch set 개수
print(f"test_dataset 개수 : {len(test_dataset)}, " + f"test_loader 개수 : {len(test_loader)}") # test_data 개수, test_loader batch set 개수
"""

'\nprint(f"train_dataset 개수 : {len(train_dataset)}, " + f"train_loader 개수 : {len(train_loader)}") # train_data 개수, train_loader batch set 개수\nprint(f"val_dataset 개수 : {len(valid_dataset)}, " + f"val_loader 개수 : {len(val_loader)}") # val_data 개수, val_loader batch set 개수\nprint(f"test_dataset 개수 : {len(test_dataset)}, " + f"test_loader 개수 : {len(test_loader)}") # test_data 개수, test_loader batch set 개수\n'

In [29]:
"""
# 데이터 분포 전처리

# 스케일러 선언
ms = MinMaxScaler()
ss = StandardScaler()

# 스케일러 적용
X_ss = ss.fit_transform(X)
y_ms = ms.fit_transform(y) 

# train, test 데이터셋 나누기
X_train = X_ss[:200, :]
X_test = X_ss[200:, :]

# train, test 데이터셋 나누기
y_train = y_ms[:200, :]
y_test = y_ms[200:, :] 

# 확인용 코드
print("Training Shape", X_train.shape, y_train.shape)
print("Testing Shape", X_test.shape, y_test.shape)
"""

'\n# 데이터 분포 전처리\n\n# 스케일러 선언\nms = MinMaxScaler()\nss = StandardScaler()\n\n# 스케일러 적용\nX_ss = ss.fit_transform(X)\ny_ms = ms.fit_transform(y) \n\n# train, test 데이터셋 나누기\nX_train = X_ss[:200, :]\nX_test = X_ss[200:, :]\n\n# train, test 데이터셋 나누기\ny_train = y_ms[:200, :]\ny_test = y_ms[200:, :] \n\n# 확인용 코드\nprint("Training Shape", X_train.shape, y_train.shape)\nprint("Testing Shape", X_test.shape, y_test.shape)\n'

In [30]:
"""
# 데이터셋 형태 전처리

#  variable : .backward 호출 시 자동계산 / list -> tensor
X_train_tensors = Variable(torch.Tensor(X_train))
X_test_tensors = Variable(torch.Tensor(X_test))

#  variable : .backward 호출 시 자동계산 / list -> tensor
y_train_tensors = Variable(torch.Tensor(y_train))
y_test_tensors = Variable(torch.Tensor(y_test))

# 데이터셋 크기 변형
X_train_tensors_f = torch.reshape(X_train_tensors,   (X_train_tensors.shape[0], 1, X_train_tensors.shape[1]))
X_test_tensors_f = torch.reshape(X_test_tensors,  (X_test_tensors.shape[0], 1, X_test_tensors.shape[1])) 

# 확인용 코드
print("Training Shape", X_train_tensors_f.shape, y_train_tensors.shape)
print("Testing Shape", X_test_tensors_f.shape, y_test_tensors.shape)
"""

'\n# 데이터셋 형태 전처리\n\n#  variable : .backward 호출 시 자동계산 / list -> tensor\nX_train_tensors = Variable(torch.Tensor(X_train))\nX_test_tensors = Variable(torch.Tensor(X_test))\n\n#  variable : .backward 호출 시 자동계산 / list -> tensor\ny_train_tensors = Variable(torch.Tensor(y_train))\ny_test_tensors = Variable(torch.Tensor(y_test))\n\n# 데이터셋 크기 변형\nX_train_tensors_f = torch.reshape(X_train_tensors,   (X_train_tensors.shape[0], 1, X_train_tensors.shape[1]))\nX_test_tensors_f = torch.reshape(X_test_tensors,  (X_test_tensors.shape[0], 1, X_test_tensors.shape[1])) \n\n# 확인용 코드\nprint("Training Shape", X_train_tensors_f.shape, y_train_tensors.shape)\nprint("Testing Shape", X_test_tensors_f.shape, y_test_tensors.shape)\n'

## 4. Model Define

### 1) 모델 정의

In [31]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(RNN, self).__init__() # 상속한 nn.Module에서 RNN에 해당하는 init 실행
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x): 
        # input x : (BATCH, LENGTH, INPUT_SIZE) 입니다 (다양한 length를 다룰 수 있습니다.).
        # 최초의 hidden state와 cell state를 초기화시켜주어야 합니다.
        # 배치 사이즈는 가변적이므로 클래스 내에선 표현하지 않습니다.
        # 만약 Bi-directional LSTM이라면 아래의 hidden and cell states의 첫번째 차원은 2*self.num_layers 입니다. 
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(args.device) # (BATCH SIZE, SEQ_LENGTH, HIDDEN_SIZE)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(args.device) # hidden state와 동일

        # LSTM 순전파
        out, _ = self.lstm(x, (h0, c0)) # output : (BATCH_SIZE, SEQ_LENGTH, HIDDEN_SIZE) tensors. (hn, cn)은 필요 없으므로 받지 않고 _로 처리합니다. 

        # 마지막 time step(sequence length)의 hidden state를 사용해 Class들의 logit을 반환합니다(hidden_size -> output_size). 
        out = self.fc(out[:, -1, :])
        return out

In [32]:
"""
class LSTM(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, num_layers, seq_length):
        super(LSTM, self).__init__()
        self.num_classes = num_classes 
        self.num_layers = num_layers 
        self.input_size = input_size 
        self.hidden_size = hidden_size 
        self.seq_length = seq_length 

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                          num_layers=num_layers, batch_first=True) 
        self.fc_1 =  nn.Linear(hidden_size, 128) 
        self.fc = nn.Linear(128, num_classes) 

        self.relu = nn.ReLU()
    
    def forward(self,x):
        h_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)) 
        c_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)) 
        
        output, (hn, cn) = self.lstm(x, (h_0, c_0)) 
        hn = hn.view(-1, self.hidden_size) 
        out = self.relu(hn)
        out = self.fc_1(out) 
        out = self.relu(out) 
        out = self.fc(out)
        return out
"""

'\nclass LSTM(nn.Module):\n    def __init__(self, num_classes, input_size, hidden_size, num_layers, seq_length):\n        super(LSTM, self).__init__()\n        self.num_classes = num_classes \n        self.num_layers = num_layers \n        self.input_size = input_size \n        self.hidden_size = hidden_size \n        self.seq_length = seq_length \n\n        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,\n                          num_layers=num_layers, batch_first=True) \n        self.fc_1 =  nn.Linear(hidden_size, 128) \n        self.fc = nn.Linear(128, num_classes) \n\n        self.relu = nn.ReLU()\n    \n    def forward(self,x):\n        h_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)) \n        c_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)) \n        \n        output, (hn, cn) = self.lstm(x, (h_0, c_0)) \n        hn = hn.view(-1, self.hidden_size) \n        out = self.relu(hn)\n        out = self.fc_1(out) \n

### 2) 모델 + 옵티마이저 + 손실함수 + 스케쥴러 + 메트릭 함수 정의

In [35]:
# Build model
model = RNN(args.input_size, args.hidden_size, args.num_layers, args.output_size).to(args.device) # 

# Build loss function
criterion = nn.MSELoss()

# Build optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)

In [ ]:
"""
# Build model
model = LSTM(num_classes, input_size, hidden_size, num_layers, X_train_tensors_f.shape[1]) 

# Build loss function
criterion = torch.nn.MSELoss()    

# Build optimizer 
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 
"""

## 5. Model Train

### 1) Load model epoch

### 2) Train model

In [62]:
for df_rpt_drop8_np_row in df_rpt_drop8_np:
    print("df_rpt_drop8_np_row.shape : ", df_rpt_drop8_np_row.shape)
    print("df_rpt_drop8_np_row : ", df_rpt_drop8_np_row)

for epoch in range(args.epochs):
    for i, df_rpt_drop8_np_row in enumerate(df_rpt_drop8_np):
        images = images.reshape(-1, sequence_length, args.input_size).to(args.device) # (BATCH(100), 1, 28, 28) -> (BATCH(100), 28, 28)
        labels = labels.to(args.device) # Size : (100)

        # 순전파
        outputs = model(images)
        loss = criterion(outputs, labels)

        # 역전파 & 최적화
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, args.epochs, i+1, total_step, loss.item()))

df_rpt_drop8_np_row.shape :  (11,)
df_rpt_drop8_np_row :  [0.    0.    1.133 0.45  0.    0.    0.    0.25  1.7   1.    0.   ]
df_rpt_drop8_np_row.shape :  (11,)
df_rpt_drop8_np_row :  [ 0.000000e+00 -1.808600e-02  1.414308e+00  4.248590e-01  3.560033e+02
  3.454337e+02  2.793784e+00  1.851300e-02  1.394299e+00  1.358461e+00
  0.000000e+00]
df_rpt_drop8_np_row.shape :  (11,)
df_rpt_drop8_np_row :  [ 0.000000e+00 -1.808600e-02  1.414308e+00  4.248590e-01  3.560033e+02
  3.454337e+02  2.793784e+00  1.851300e-02  1.394299e+00  1.358461e+00
  0.000000e+00]
df_rpt_drop8_np_row.shape :  (11,)
df_rpt_drop8_np_row :  [ 0.000000e+00 -1.808600e-02  1.414308e+00  4.248590e-01  3.560033e+02
  3.454337e+02  2.793784e+00  1.851300e-02  1.394299e+00  1.358461e+00
  0.000000e+00]
df_rpt_drop8_np_row.shape :  (11,)
df_rpt_drop8_np_row :  [ 0.000000e+00 -1.808600e-02  1.414308e+00  4.248590e-01  3.560033e+02
  3.454337e+02  2.793784e+00  1.851300e-02  1.394299e+00  1.358461e+00
  0.000000e+00]
df_rpt_dro

df_rpt_drop8_np_row.shape :  (11,)
df_rpt_drop8_np_row :  [ 0.000000e+00  2.860500e-02  1.445271e+00 -1.765910e-02  5.573426e+00
  3.567543e+02  6.432225e+00  6.346490e-02  1.494679e+00  8.581256e-01
  0.000000e+00]
df_rpt_drop8_np_row.shape :  (11,)
df_rpt_drop8_np_row :  [ 0.000000e+00  3.439570e-02  1.447228e+00 -1.960650e-02  7.760980e+00
  3.557799e+02  5.266164e+00  5.915330e-02  1.485469e+00  8.397339e-01
  0.000000e+00]
df_rpt_drop8_np_row.shape :  (11,)
df_rpt_drop8_np_row :  [ 0.000000e+00  3.963070e-02  1.447627e+00 -2.177230e-02  9.343156e+00
  3.554879e+02  3.961205e+00  7.338910e-02  1.488897e+00  8.137166e-01
  0.000000e+00]
df_rpt_drop8_np_row.shape :  (11,)
df_rpt_drop8_np_row :  [ 0.000000e+00  4.368380e-02  1.447452e+00 -2.270410e-02  9.998158e+00
  3.554749e+02  2.751610e+00  9.450670e-02  1.486115e+00  8.007719e-01
  0.000000e+00]
df_rpt_drop8_np_row.shape :  (11,)
df_rpt_drop8_np_row :  [ 0.000000e+00  4.814480e-02  1.446951e+00 -2.295020e-02  1.102467e+01
  3.557

df_rpt_drop8_np_row.shape :  (11,)
df_rpt_drop8_np_row :  [ 7.000000e+00  1.578600e-03  1.451727e+00 -3.563370e-02  5.780492e-01
  3.516843e+02  4.902543e+00 -1.593970e-02  1.404553e+00  8.628187e-01
  0.000000e+00]
df_rpt_drop8_np_row.shape :  (11,)
df_rpt_drop8_np_row :  [ 7.000000e+00  8.411000e-04  1.451518e+00 -3.496470e-02  6.592385e-01
  3.518186e+02  4.778417e+00 -1.798220e-02  1.406674e+00  8.626541e-01
  0.000000e+00]
df_rpt_drop8_np_row.shape :  (11,)
df_rpt_drop8_np_row :  [ 7.000000e+00 -9.305000e-04  1.451522e+00 -3.394260e-02  3.827865e-01
  3.518698e+02  5.113340e+00 -2.391700e-02  1.407842e+00  8.638805e-01
  0.000000e+00]
df_rpt_drop8_np_row.shape :  (11,)
df_rpt_drop8_np_row :  [ 7.000000e+00 -2.786200e-03  1.450811e+00 -3.318360e-02  3.599658e+02
  3.521877e+02  5.385667e+00 -2.312610e-02  1.413909e+00  8.639447e-01
  0.000000e+00]
df_rpt_drop8_np_row.shape :  (11,)
df_rpt_drop8_np_row :  [ 7.000000e+00 -3.785400e-03  1.450832e+00 -3.264580e-02  3.599542e+02
  3.521

ValueError: too many values to unpack (expected 2)

In [ ]:
"""
for epoch in range(num_epochs):
    outputs = model.forward(X_train_tensors_f) 
    optimizer.zero_grad()  
    loss = criterion(outputs, y_train_tensors) 
    loss.backward() 
    optimizer.step() 
    if epoch % 100 == 0:
        print("Epoch: %d, loss: %1.5f" % (epoch, loss.item())) 
"""